In [ ]:
import pandas as pd
import numpy as np
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
import os

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# ==============================================================================
# 1. LOAD AND PREPARE DATA
# ==============================================================================
try:
    df = pd.read_csv('india_complete_Full_tourism_dataset.csv')
    print("✓ Dataset loaded successfully.")
except FileNotFoundError:
    print("✗ Error: 'india_complete_tourism_dataset.csv' not found.")
    exit()

# Clean data
df['Cost'] = pd.to_numeric(df['Cost'], errors='coerce')
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df.dropna(subset=['Cost', 'Rating'], inplace=True)
df = df.reset_index(drop=True)

# Setup categories
CATEGORIES = ['Nature & Relaxation', 'Culture & Heritage', 'Shopping',
              'Food & Culinary', 'Hotels & Stay', 'Adventure']
CATEGORY_MAP = {cat: i for i, cat in enumerate(CATEGORIES)}
NUM_CATEGORIES = len(CATEGORIES)

print(f"✓ Total items in dataset: {len(df)}")
print(f"✓ Available locations: {df['Location'].nunique()}")
print(f"✓ Categories: {CATEGORIES}\n")

# ==============================================================================
# 2. SMART TOURISM ENVIRONMENT
# ==============================================================================
class SmartTourismEnv:
    def __init__(self, data, category_map):
        self.data = data
        self.category_map = category_map
        self.reset_episode_vars()

    def reset_episode_vars(self):
        self.package = []
        self.current_cost = 0
        self.selected_categories = set()
        self.location_items = pd.DataFrame()
        self.available_items = []

    def reset(self, budget, location, num_days, categories):
        """Reset environment for new episode"""
        self.budget = budget
        self.initial_budget = budget
        self.location = location
        self.num_days = num_days
        self.required_categories = set(categories)
        self.num_required_categories = len(categories)

        self.reset_episode_vars()

        # Filter items for the specific location
        self.location_items = self.data[self.data['Location'] == location].reset_index(drop=True)

        if self.location_items.empty:
            return None  # Location not found

        # Build available items list with indices
        self.available_items = list(range(len(self.location_items)))

        # Calculate action space: each item + stop action
        self.action_space_size = len(self.location_items) + 1
        self.stop_action = len(self.location_items)

        return self._get_state()

    def _get_state(self):
        """Encode current state"""
        # Budget utilization
        budget_used_ratio = self.current_cost / self.initial_budget if self.initial_budget > 0 else 0
        budget_remaining_ratio = (self.initial_budget - self.current_cost) / self.initial_budget if self.initial_budget > 0 else 0

        # Category coverage
        categories_covered = len(self.selected_categories)
        categories_coverage_ratio = categories_covered / self.num_required_categories if self.num_required_categories > 0 else 0

        # Required categories vector (one-hot encoding)
        required_vector = np.zeros(NUM_CATEGORIES)
        for cat in self.required_categories:
            if cat in self.category_map:
                required_vector[self.category_map[cat]] = 1

        # Selected categories vector
        selected_vector = np.zeros(NUM_CATEGORIES)
        for cat in self.selected_categories:
            if cat in self.category_map:
                selected_vector[self.category_map[cat]] = 1

        # Package size
        package_size = len(self.package)
        has_hotel = 1 if 'Hotels & Stay' in self.selected_categories else 0

        # Combine all features
        state = np.concatenate([
            [budget_used_ratio, budget_remaining_ratio, categories_coverage_ratio,
             package_size / 10.0, has_hotel, self.num_days / 10.0],
            required_vector,
            selected_vector
        ])

        return state.reshape(1, -1)

    def _calculate_hotel_cost(self, per_night_cost):
        """Calculate total hotel cost for the trip duration"""
        return per_night_cost * self.num_days

    def step(self, action):
        """Execute action and return next state, reward, done"""
        reward = 0
        done = False

        # STOP ACTION
        if action == self.stop_action:
            done = True
            # Evaluate final package
            has_hotel = 'Hotels & Stay' in self.selected_categories
            categories_covered = len(self.selected_categories)

            # Check if package is valid
            if len(self.package) == 0:
                reward = -100  # No items selected
            elif 'Hotels & Stay' in self.required_categories and not has_hotel:
                reward = -80  # Missing required hotel
            elif categories_covered < self.num_required_categories:
                # Missing some categories
                missing = self.num_required_categories - categories_covered
                reward = -50 - (missing * 15)
            else:
                # Valid package - reward based on quality
                budget_utilization = self.current_cost / self.initial_budget
                category_bonus = categories_covered * 20

                # Optimal utilization: 70-95% of budget
                if 0.7 <= budget_utilization <= 0.95:
                    utilization_bonus = 50
                elif 0.5 <= budget_utilization < 0.7:
                    utilization_bonus = 30
                else:
                    utilization_bonus = 10

                avg_rating = np.mean([item['Rating'] for item in self.package])
                rating_bonus = avg_rating * 5

                reward = 100 + category_bonus + utilization_bonus + rating_bonus

            return self._get_state(), reward, done, {}

        # SELECT ITEM ACTION
        if action >= len(self.location_items):
            return self._get_state(), -100, True, {}  # Invalid action

        if action not in self.available_items:
            return self._get_state(), -30, False, {}  # Already selected

        item = self.location_items.iloc[action]
        item_category = item['Category']

        # Calculate actual cost
        if item['Type'] == 'Hotel':
            actual_cost = self._calculate_hotel_cost(item['Cost'])

            # Check if hotel already selected
            if 'Hotels & Stay' in self.selected_categories:
                return self._get_state(), -40, False, {}  # Duplicate hotel
        else:
            actual_cost = item['Cost']

        # Check budget constraint
        if self.current_cost + actual_cost > self.budget:
            return self._get_state(), -25, False, {}  # Exceeds budget

        # Valid selection - add to package
        self.available_items.remove(action)
        self.current_cost += actual_cost
        self.selected_categories.add(item_category)

        package_item = item.to_dict()
        package_item['ActualCost'] = actual_cost
        if item['Type'] == 'Hotel':
            package_item['Nights'] = self.num_days
        self.package.append(package_item)

        # Calculate reward
        reward = 10  # Base reward for valid selection

        # Category bonus
        if item_category in self.required_categories:
            reward += 25  # Strong bonus for required category
        else:
            reward -= 5  # Small penalty for non-required

        # Rating bonus
        reward += item['Rating'] * 2

        # Budget distribution bonus
        expected_budget_per_category = self.initial_budget / self.num_required_categories
        if actual_cost <= expected_budget_per_category * 1.5:
            reward += 10  # Good budget distribution
        else:
            reward -= 10  # Taking too much budget

        # Progress bonus
        if len(self.selected_categories) == self.num_required_categories:
            reward += 30  # Covered all categories

        # Check if episode should end naturally
        affordable_items = [i for i in self.available_items
                          if self.location_items.iloc[i]['Cost'] <= (self.budget - self.current_cost)]

        if not affordable_items and len(self.selected_categories) >= self.num_required_categories:
            done = True
            reward += 20  # Bonus for completing successfully

        return self._get_state(), reward, done, {}

# ==============================================================================
# 3. DQN AGENT
# ==============================================================================
class DQNAgent:
    def __init__(self, state_size, max_action_size):
        self.state_size = state_size
        self.action_size = max_action_size
        self.memory = deque(maxlen=3000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.0005
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential([
            Dense(128, input_dim=self.state_size, activation='relu'),
            Dropout(0.2),
            Dense(128, activation='relu'),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='huber', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, valid_actions):
        """Select action using epsilon-greedy policy"""
        if np.random.rand() <= self.epsilon:
            return random.choice(valid_actions)

        q_values = self.model.predict(state, verbose=0)[0]
        valid_q = [(action, q_values[action]) for action in valid_actions]
        return max(valid_q, key=lambda x: x[1])[0]

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)
        states = np.vstack([s for s, _, _, _, _ in minibatch])
        next_states = np.vstack([ns for _, _, _, ns, _ in minibatch])

        current_q = self.model.predict(states, verbose=0)
        next_q = self.model.predict(next_states, verbose=0)

        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            if done:
                current_q[i][action] = reward
            else:
                current_q[i][action] = reward + self.gamma * np.max(next_q[i])

        self.model.fit(states, current_q, epochs=1, verbose=0, batch_size=batch_size)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self, filename):
        self.model.save(filename)

    def load(self, filename):
        self.model.load_weights(filename)

# ==============================================================================
# 4. TRAINING
# ==============================================================================
def train_agent(episodes=3000):
    print("\n" + "="*70)
    print("TRAINING DQN AGENT FOR SMART TOURISM RECOMMENDATIONS")
    print("="*70)

    # Calculate state size
    state_size = 6 + (NUM_CATEGORIES * 2)  # Features + required + selected categories
    max_action_size = len(df) + 1

    agent = DQNAgent(state_size, max_action_size)
    env = SmartTourismEnv(df, CATEGORY_MAP)

    batch_size = 64
    locations = df['Location'].unique()

    for episode in range(episodes):
        # Random scenario
        location = random.choice(locations)
        budget = random.randint(3000, 25000)
        num_days = random.randint(1, 5)

        # Select random categories (1-4 categories)
        num_cats = random.randint(1, min(4, len(CATEGORIES)))

        # Ensure Hotels & Stay is included 80% of the time for multi-day trips
        if num_days > 1 and random.random() < 0.8:
            cats = ['Hotels & Stay']
            remaining = [c for c in CATEGORIES if c != 'Hotels & Stay']
            cats.extend(random.sample(remaining, num_cats - 1))
        else:
            cats = random.sample(CATEGORIES, num_cats)

        state = env.reset(budget, location, num_days, cats)

        if state is None:
            continue

        total_reward = 0
        steps = 0
        max_steps = 20

        for step in range(max_steps):
            valid_actions = env.available_items + [env.stop_action]
            action = agent.act(state, valid_actions)

            next_state, reward, done, _ = env.step(action)
            total_reward += reward

            agent.remember(state, action, reward, next_state, done)
            state = next_state
            steps += 1

            if done:
                break

        agent.replay(batch_size)

        if (episode + 1) % 50 == 0:
            coverage = len(env.selected_categories) / len(cats) * 100 if cats else 0
            print(f"Ep {episode+1:4d}/{episodes} | {location[:15]:15s} | "
                  f"Budget: ₹{budget:5d} | Days: {num_days} | "
                  f"Reward: {total_reward:6.1f} | Coverage: {coverage:.0f}% | "
                  f"ε: {agent.epsilon:.3f}")

    print("\n✓ Training completed successfully!")
    return agent

# ==============================================================================
# 5. RECOMMENDATION FUNCTION
# ==============================================================================
def recommend_package(agent, budget, location, num_days, categories):
    """Generate recommendation for user input"""
    print("\n" + "="*70)
    print(f"GENERATING SMART PACKAGE FOR {location.upper()}")
    print("="*70)
    print(f"Budget: ₹{budget:,} | Duration: {num_days} day(s)")
    print(f"Preferred Categories: {', '.join(categories)}")
    print("-"*70)

    # Check if location exists
    if location not in df['Location'].values:
        print(f"\n✗ Sorry, '{location}' is not available in our database.")
        print(f"  Available locations: {', '.join(sorted(df['Location'].unique()))}")
        return

    agent.epsilon = 0.0  # Use learned policy only
    env = SmartTourismEnv(df, CATEGORY_MAP)
    state = env.reset(budget, location, num_days, categories)

    # Run agent
    for _ in range(20):
        valid_actions = env.available_items + [env.stop_action]
        action = agent.act(state, valid_actions)
        next_state, _, done, _ = env.step(action)
        state = next_state
        if done:
            break

    # Check if valid package generated
    package = env.package

    if not package:
        print("\n✗ No suitable package found with the given budget and requirements.")
        print("  Suggestions:")
        print("  • Try increasing your budget")
        print("  • Select fewer categories")
        print("  • Check if the location has items in your preferred categories")
        return

    # Check completeness
    has_hotel = 'Hotels & Stay' in env.selected_categories
    missing_cats = [cat for cat in categories if cat not in env.selected_categories]

    if 'Hotels & Stay' in categories and not has_hotel:
        print("\n⚠ WARNING: Could not find an affordable hotel within budget!")
        print("  Consider increasing budget or reducing trip duration.")
        return

    if missing_cats and len(missing_cats) == len(categories):
        print(f"\n✗ No items found for any selected category within budget.")
        return

    # Display package
    print(f"\n✓ Smart Package Generated Successfully!")
    print(f"\n{'Item':<35} {'Category':<25} {'Details':<15} {'Cost':>12}")
    print("-"*90)

    total_cost = 0
    for item in package:
        name = item['Name'][:33]
        category = item['Category'][:23]

        if item['Type'] == 'Hotel':
            details = f"{item['Nights']} night(s)"
        else:
            details = f"Rating: {item['Rating']:.1f}"

        cost = item['ActualCost']
        total_cost += cost

        print(f"{name:<35} {category:<25} {details:<15} ₹{cost:>10,.0f}")

    print("-"*90)
    print(f"{'TOTAL COST':<77} ₹{total_cost:>10,.0f}")
    print(f"{'REMAINING BUDGET':<77} ₹{budget - total_cost:>10,.0f}")
    print(f"{'BUDGET UTILIZATION':<77} {total_cost/budget*100:>10.1f}%")

    # Coverage summary
    print(f"\n📊 Category Coverage: {len(env.selected_categories)}/{len(categories)}")
    if missing_cats:
        print(f"⚠ Missing categories: {', '.join(missing_cats)}")
        print("  (No affordable options found for these categories)")

    print("="*70 + "\n")

# ==============================================================================
# 6. MAIN EXECUTION
# ==============================================================================
MODEL_FILE = 'smart_tourism_dqn.h5'

# Train or load model
if os.path.exists(MODEL_FILE):
    print(f"\n✓ Loading pre-trained model: {MODEL_FILE}")
    state_size = 6 + (NUM_CATEGORIES * 2)
    max_action_size = len(df) + 1
    agent = DQNAgent(state_size, max_action_size)
    agent.load(MODEL_FILE)
    agent.epsilon = 0.0
else:
    print("\n⚠ No pre-trained model found. Starting training...")
    agent = train_agent(episodes=3000)
    agent.save(MODEL_FILE)
    print(f"✓ Model saved as: {MODEL_FILE}")

# Test scenarios
print("\n\n" + "="*70)
print("TESTING SMART RECOMMENDATIONS")
print("="*70)

# Scenario 1: Multi-category with good budget
recommend_package(
    agent,
    budget=10000,
    location='Ooty',
    num_days=2,
    categories=['Nature & Relaxation', 'Adventure', 'Hotels & Stay']
)

# Scenario 2: Hotel only
recommend_package(
    agent,
    budget=8000,
    location='Ooty',
    num_days=3,
    categories=['Hotels & Stay']
)

# Scenario 3: Low budget challenge
recommend_package(
    agent,
    budget=5000,
    location='Ooty',
    num_days=2,
    categories=['Nature & Relaxation', 'Adventure', 'Hotels & Stay']
)

# Scenario 4: Multiple categories
recommend_package(
    agent,
    budget=15000,
    location='Mysore',
    num_days=3,
    categories=['Culture & Heritage', 'Food & Culinary', 'Hotels & Stay', 'Shopping']
)

✓ Dataset loaded successfully.
✓ Total items in dataset: 645
✓ Available locations: 86
✓ Categories: ['Nature & Relaxation', 'Culture & Heritage', 'Shopping', 'Food & Culinary', 'Hotels & Stay', 'Adventure']


⚠ No pre-trained model found. Starting training...

TRAINING DQN AGENT FOR SMART TOURISM RECOMMENDATIONS
Ep   50/3000 | Serenity Beach  | Budget: ₹19991 | Days: 5 | Reward:   81.2 | Coverage: 100% | ε: 0.835
Ep  100/3000 | Palolem         | Budget: ₹21565 | Days: 5 | Reward: -106.8 | Coverage: 67% | ε: 0.650
Ep  150/3000 | White Town      | Budget: ₹19784 | Days: 2 | Reward:  535.4 | Coverage: 200% | ε: 0.506
Ep  200/3000 | Sivasagar       | Budget: ₹20089 | Days: 2 | Reward:  296.9 | Coverage: 100% | ε: 0.394
Ep  250/3000 | Gaya            | Budget: ₹ 3986 | Days: 5 | Reward:  -25.8 | Coverage: 50% | ε: 0.306
Ep  300/3000 | Pondicherry     | Budget: ₹15093 | Days: 1 | Reward:   76.2 | Coverage: 75% | ε: 0.238
Ep  350/3000 | Amaravati       | Budget: ₹21994 | Days: 4 | Reward:  3

Ep 3000/3000 | Serenity Beach  | Budget: ₹12205 | Days: 5 | Reward:  106.2 | Coverage: 75% | ε: 0.010

✓ Training completed successfully!
✓ Model saved as: smart_tourism_dqn.h5


TESTING SMART RECOMMENDATIONS

GENERATING SMART PACKAGE FOR OOTY
Budget: ₹10,000 | Duration: 2 day(s)
Preferred Categories: Nature & Relaxation, Adventure, Hotels & Stay
----------------------------------------------------------------------

⚠ WARNING: Could not find an affordable hotel within budget!
  Consider increasing budget or reducing trip duration.

GENERATING SMART PACKAGE FOR OOTY
Budget: ₹8,000 | Duration: 3 day(s)
Preferred Categories: Hotels & Stay
----------------------------------------------------------------------

⚠ WARNING: Could not find an affordable hotel within budget!
  Consider increasing budget or reducing trip duration.

GENERATING SMART PACKAGE FOR OOTY
Budget: ₹5,000 | Duration: 2 day(s)
Preferred Categories: Nature & Relaxation, Adventure, Hotels & Stay
---------------------------